In [1]:
# Preliminary setup of execution environment
import os
from pathlib import Path
import subprocess

nntile_dir = Path.cwd() / ".."

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Limit CUDA visibility
os.environ["OMP_NUM_THREADS"] = "1" # Disable BLAS parallelism
os.environ["PYTHONPATH"] = str(nntile_dir / "build" / "wrappers" / "python") # Path to a binary dir of NNTile Python wrappers

# All StarPU environment variables are available at https://files.inria.fr/starpu/doc/html/ExecutionConfigurationThroughEnvironmentVariables.html
os.environ["STARPU_NCPU"] = "1" # Use only 1 CPU core
os.environ["STARPU_NCUDA"] = "1" # Use only 1 CUDA device
os.environ["STARPU_SILENT"] = "1" # Do not show lots of StarPU outputs
os.environ["STARPU_SCHED"] = "dmdasd" # Name StarPU scheduler to be used
os.environ["STARPU_FXT_TRACE"] = "0" # Do not generate FXT traces
os.environ["STARPU_WORKERS_NOBIND"] = "1" # Do not bind workers (it helps if several instances of StarPU run in parallel)
os.environ["STARPU_PROFILING"] = "1" # This enables logging performance of workers and bandwidth of memory nodes
os.environ["STARPU_HOME"] = str(Path.cwd() / "starpu") # Main directory in which StarPU stores its configuration files
os.environ["STARPU_PERF_MODEL_DIR"] = str(Path(os.environ["STARPU_HOME"]) / "sampling") # Main directory in which StarPU stores its performance model files
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CPU"] = "1" # Assume all CPU cores are equal
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CUDA"] = "1" # Assume all CUDA devices are equal
os.environ["STARPU_HOSTNAME"] = "GPT2_example" # Force the hostname to be used when managing performance model files
os.environ["STARPU_FXT_PREFIX"] = str(Path(os.environ["STARPU_HOME"]) / "fxt") # Directory to store FXT traces if enabled

# NNTile-related
os.environ["NNTILE_LOGGER"] = "1" # Enable logger
os.environ["NNTILE_LOGGER_SERVER_ADDR"] = "127.0.0.1" # Logger will be running on the localhost
os.environ["NNTILE_LOGGER_SERVER_PORT"] = "5001" # Port for logger server
os.environ["NNTILE_LOGGER_CLIENT_PORT"] = "6006" # Port for client web interface of the logger
os.environ["NNTILE_LOGGER_SERVER_DIR"] = str(Path.cwd() / "logs") # Directory to store logs on the logger server

In [2]:
# Launch logger if needed
if os.getenv("NNTILE_LOGGER", "0") != "0":
    logger_env = os.environ.copy()
    logger_env.update({
        "LOG_DIR": os.getenv("NNTILE_LOGGER_SERVER_DIR"),
        "SPLIT_HOURS": "720",
        "CLEAR_LOGS": "0",
        "SERVER_PORT": os.getenv("NNTILE_LOGGER_SERVER_PORT")
    })
    logger_proc = subprocess.Popen(["python", nntile_dir / "logger" / "server.py"], env=logger_env)

2024-08-09 17:47:26.334610: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 17:47:26.369313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Prepare TinyStories dataset into train.bin file
!python ../wrappers/python/examples/causal_lm_data_preparation.py --seq-len=1024 --batch-size=2

Repo card metadata block was not found. Setting CardData to empty.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
# Launch an external python process to finetune a pretrained LLaMa model on TinyStories
# If logger server is launched, then TensorBoard results can be accessed at localhost:6006
!python ../wrappers/python/examples/llama_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/llama_1.3b_config.json" \
    --optimizer="adam" --lr=1e-4 --dtype=bf16 --nepochs=100 --batch-size=2 --minibatch-size=1 \
    --dataset-file="tinystories/train.bin" --logger --logger-server-addr=127.0.0.1

Namespace(remote_model_name='kimihailv/llama-1.3b', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/llama_1.3b_config.json', save_checkpoint_path='.model', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=2, minibatch_size=1, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='bf16', restrict='cuda', flashattention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=100, logger=True, logger_server_addr='127.0.0.1', logger_server_port=5001)
LlamaConfig {
  "activation_function": "silu",
  "architectures": [
    "LlamaCasualForLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "fp32",
  "eos_token_id": 2,
  "flashattention": false,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5504,
  "max_position_embeddings": 4096,
  "mlp_bias": fals